# Noel Perakende Satis Analizi ve Makine Ogrenmesi Projesi

Bu projede, Noel donemi perakende satis verilerini analiz edecegiz. Veri bilimi surecinin tum asamalarini uygulayacagiz:

1. Veri Yukleme ve Kesifsel Veri Analizi (EDA)
2. Veri Birlestirme ve Temizleme
3. Istatistiksel Analizler
4. Gorsellestirmeler
5. Veri On Isleme
6. Makine Ogrenmesi Modelleri
7. Model Degerlendirme

**Veri Seti:** Christmas Retail Sales and Marketing Analysis Dataset

## 1. Kutuphanelerin Yuklenmesi

Projede kullanacagimiz temel kutuphaneleri yukluyoruz. Bu kutuphaneler veri bilimi calismalarinin temelini olusturmaktadir.

In [ ]:
# Temel kutuphaneler
import numpy as np
import pandas as pd

# Gorsellestirme kutuphaneleri
import matplotlib.pyplot as plt
import seaborn as sns

# Istatistiksel testler icin
from scipy import stats
from scipy.stats import shapiro, ttest_ind, f_oneway, pearsonr, spearmanr

# Makine ogrenmesi icin sklearn kutuphanesi
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score, roc_curve

# Siniflandirma modelleri
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Uyarilari kapatiyoruz
import warnings
warnings.filterwarnings('ignore')

# Gorsellestirme ayarlari
plt.rcParams['figure.figsize'] = (10, 6)
sns.set_style('whitegrid')

print("Kutuphaneler basariyla yuklendi!")

## 2. Verilerin Yuklenmesi

Excel dosyasindaki tum tablolari yukluyoruz. Veri setimiz 7 farkli tablodan olusmaktadir:
- **OrderLine**: Siparis satir detaylari (urun, miktar, fiyat)
- **Product**: Urun bilgileri (kategori, maliyet, fiyat)
- **Promotion**: Promosyon kampanyalari
- **Returns**: Iade bilgileri
- **Calendar**: Takvim ve tatil bilgileri
- **Fulfillment**: Kargo ve teslimat bilgileri
- **OrderHeader**: Siparis baslik bilgileri

In [ ]:
# Excel dosyasini okuyoruz
dosya_yolu = 'Christmas_Retail_Sales_and_Marketing_Analysis_Dataset.xlsx'

# Tum sayfalari okuyoruz
order_line = pd.read_excel(dosya_yolu, sheet_name='OrderLine')
product = pd.read_excel(dosya_yolu, sheet_name='Product')
promotion = pd.read_excel(dosya_yolu, sheet_name='Promotion')
returns = pd.read_excel(dosya_yolu, sheet_name='Returns')
calendar = pd.read_excel(dosya_yolu, sheet_name='Calendar')
fulfillment = pd.read_excel(dosya_yolu, sheet_name='Fulfillment')
order_header = pd.read_excel(dosya_yolu, sheet_name='OrderHeader')

print("Tum tablolar basariyla yuklendi!")

### 2.1 Tablolarin Incelenmesi

Her tablonun boyutunu, sutunlarini ve veri tiplerini inceliyoruz.

In [ ]:
# Tum tablolari bir sozlukte topluyoruz
tablolar = {
    'OrderLine': order_line,
    'Product': product,
    'Promotion': promotion,
    'Returns': returns,
    'Calendar': calendar,
    'Fulfillment': fulfillment,
    'OrderHeader': order_header
}

# Her tablonun boyutunu yazdiriyoruz
print("=" * 50)
print("TABLO BOYUTLARI")
print("=" * 50)
for tablo_adi, tablo in tablolar.items():
    print(f"{tablo_adi}: {tablo.shape[0]} satir, {tablo.shape[1]} sutun")

In [ ]:
# OrderLine tablosunun ilk 5 satirini inceleyelim
print("ORDER LINE TABLOSU")
print("=" * 50)
order_line.head()

In [ ]:
# Product tablosunu inceleyelim
print("PRODUCT TABLOSU")
print("=" * 50)
product.head()

In [ ]:
# OrderHeader tablosunu inceleyelim
print("ORDER HEADER TABLOSU")
print("=" * 50)
order_header.head()

In [ ]:
# Fulfillment tablosunu inceleyelim
print("FULFILLMENT TABLOSU")
print("=" * 50)
fulfillment.head()

## 3. Veri Birlestirme (Merge Islemleri)

Pandas'in merge fonksiyonunu kullanarak tablolari birlestiriyoruz. Bu islem SQL'deki JOIN islemine benzer.

In [ ]:
# OrderLine ve Product tablolarini birlestiriyoruz
# ProductID uzerinden left join yapiyoruz
df = pd.merge(order_line, product, on='ProductID', how='left')
print(f"OrderLine + Product birlestirme: {df.shape}")

# OrderHeader tablosunu ekliyoruz
df = pd.merge(df, order_header, on='OrderID', how='left')
print(f"+ OrderHeader birlestirme: {df.shape}")

# Fulfillment tablosunu ekliyoruz
df = pd.merge(df, fulfillment, on='OrderID', how='left')
print(f"+ Fulfillment birlestirme: {df.shape}")

print("\nAna veri seti olusturuldu!")

In [ ]:
# Birlestirilmis veri setinin yapisini inceleyelim
print("BIRLESTIRILMIS VERI SETI")
print("=" * 50)
print(f"Toplam satir sayisi: {df.shape[0]}")
print(f"Toplam sutun sayisi: {df.shape[1]}")
print("\nSutunlar:")
print(df.columns.tolist())

In [ ]:
# Veri setinin ilk 5 satiri
df.head()

In [ ]:
# Veri tiplerini inceleyelim
print("VERI TIPLERI")
print("=" * 50)
df.dtypes

## 4. Kesifsel Veri Analizi (EDA)

Veri setinin genel ozelliklerini, istatistiklerini ve dagilimlarini inceliyoruz.

In [ ]:
# Temel istatistikler - describe() fonksiyonu
# Sayisal degiskenler icin ortalama, standart sapma, min, max gibi degerleri gosterir
print("SAYISAL DEGISKENLERIN ISTATISTIKLERI")
print("=" * 50)
df.describe()

In [ ]:
# Kategorik degiskenlerin istatistikleri
print("KATEGORIK DEGISKENLERIN ISTATISTIKLERI")
print("=" * 50)
df.describe(include='object')

### 4.1 Eksik Veri Analizi

Eksik verileri tespit etmek veri temizliginin onemli bir adimidir.

In [ ]:
# Eksik veri sayisi ve orani
eksik_veri = pd.DataFrame({
    'Eksik Sayisi': df.isnull().sum(),
    'Eksik Orani (%)': (df.isnull().sum() / len(df) * 100).round(2)
})

# Sadece eksik verisi olan sutunlari gosterelim
eksik_veri = eksik_veri[eksik_veri['Eksik Sayisi'] > 0].sort_values('Eksik Sayisi', ascending=False)

print("EKSIK VERI ANALIZI")
print("=" * 50)
if len(eksik_veri) == 0:
    print("Veri setinde eksik veri bulunmamaktadir.")
else:
    print(eksik_veri)

### 4.2 Temel Istatistikler

Merkezi egilim olculeri (ortalama, medyan, mod) ve dagilim olculerini (standart sapma, varyans, carpiklik, basiklik) hesaplayacagiz.

In [ ]:
# LineRevenue$ (Satis Geliri) sutunu icin istatistikler
gelir = df['LineRevenue$']

print("SATIS GELIRI ISTATISTIKLERI")
print("=" * 50)
print(f"Ortalama (Mean): ${gelir.mean():,.2f}")
print(f"Medyan (Median): ${gelir.median():,.2f}")
print(f"Mod (Mode): ${gelir.mode()[0]:,.2f}")
print(f"Standart Sapma (Std): ${gelir.std():,.2f}")
print(f"Varyans (Var): ${gelir.var():,.2f}")
print(f"Minimum: ${gelir.min():,.2f}")
print(f"Maksimum: ${gelir.max():,.2f}")
print(f"Carpiklik (Skewness): {gelir.skew():.4f}")
print(f"Basiklik (Kurtosis): {gelir.kurtosis():.4f}")

In [ ]:
# Ceyreklikler (Quartiles)
print("\nCEYREKLIKLER")
print("=" * 50)
print(f"Q1 (25%): ${gelir.quantile(0.25):,.2f}")
print(f"Q2 (50%): ${gelir.quantile(0.50):,.2f}")
print(f"Q3 (75%): ${gelir.quantile(0.75):,.2f}")
print(f"IQR (Q3-Q1): ${gelir.quantile(0.75) - gelir.quantile(0.25):,.2f}")

## 5. Gorsellestirmeler

Seaborn ve Matplotlib kullanarak verilerimizi gorsellestiriyoruz.

### 5.1 Kategorik Degisken Gorsellestirmeleri

In [ ]:
# Urun kategorilerinin dagilimi
plt.figure(figsize=(12, 6))
kategori_sayilari = df['Category'].value_counts()
sns.barplot(x=kategori_sayilari.index, y=kategori_sayilari.values, palette='viridis')
plt.title('Urun Kategorilerinin Dagilimi', fontsize=14)
plt.xlabel('Kategori')
plt.ylabel('Satis Adedi')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Odeme tipi dagilimi - countplot kullanimi
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='PaymentType', palette='Set2')
plt.title('Odeme Tipi Dagilimi', fontsize=14)
plt.xlabel('Odeme Tipi')
plt.ylabel('Sayi')
plt.tight_layout()
plt.show()

In [ ]:
# Teslimat durumu dagilimi - pie chart
plt.figure(figsize=(10, 8))
teslimat_durumu = df['DeliveryStatus'].value_counts()
plt.pie(teslimat_durumu.values, labels=teslimat_durumu.index, autopct='%1.1f%%', 
        colors=sns.color_palette('pastel'), startangle=90)
plt.title('Teslimat Durumu Dagilimi', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Kargo firmasi dagilimi
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Carrier', palette='husl')
plt.title('Kargo Firmasi Dagilimi', fontsize=14)
plt.xlabel('Kargo Firmasi')
plt.ylabel('Siparis Sayisi')
plt.tight_layout()
plt.show()

### 5.2 Sayisal Degisken Gorsellestirmeleri

In [ ]:
# Satis geliri dagilimi - histogram
plt.figure(figsize=(12, 6))
sns.histplot(df['LineRevenue$'], kde=True, bins=50, color='steelblue')
plt.title('Satis Geliri Dagilimi', fontsize=14)
plt.xlabel('Gelir ($)')
plt.ylabel('Frekans')

# Ortalama ve medyan cizgileri
plt.axvline(df['LineRevenue$'].mean(), color='red', linestyle='--', label=f'Ortalama: ${df["LineRevenue$"].mean():,.2f}')
plt.axvline(df['LineRevenue$'].median(), color='green', linestyle='--', label=f'Medyan: ${df["LineRevenue$"].median():,.2f}')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Birim fiyat dagilimi
plt.figure(figsize=(12, 6))
sns.histplot(df['UnitPrice'], kde=True, bins=50, color='coral')
plt.title('Birim Fiyat Dagilimi', fontsize=14)
plt.xlabel('Birim Fiyat ($)')
plt.ylabel('Frekans')
plt.tight_layout()
plt.show()

In [ ]:
# Miktar dagilimi
plt.figure(figsize=(10, 6))
sns.histplot(df['Qty'], kde=False, bins=10, color='purple')
plt.title('Siparis Miktari Dagilimi', fontsize=14)
plt.xlabel('Miktar')
plt.ylabel('Frekans')
plt.tight_layout()
plt.show()

### 5.3 Boxplot Gorsellestirmeleri

Boxplot (kutu grafigi), verilerin dagilimini ve aykiri degerleri gormek icin kullanilir.

In [ ]:
# Kategorilere gore gelir boxplot
plt.figure(figsize=(14, 6))
sns.boxplot(data=df, x='Category', y='LineRevenue$', palette='Set3')
plt.title('Kategorilere Gore Satis Geliri Dagilimi', fontsize=14)
plt.xlabel('Kategori')
plt.ylabel('Gelir ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Kargo firmasina gore kargo ucreti boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='Carrier', y='ShipCost$', palette='coolwarm')
plt.title('Kargo Firmasina Gore Kargo Ucreti Dagilimi', fontsize=14)
plt.xlabel('Kargo Firmasi')
plt.ylabel('Kargo Ucreti ($)')
plt.tight_layout()
plt.show()

In [ ]:
# Odeme tipine gore gelir violin plot
plt.figure(figsize=(12, 6))
sns.violinplot(data=df, x='PaymentType', y='LineRevenue$', palette='muted')
plt.title('Odeme Tipine Gore Satis Geliri Dagilimi (Violin Plot)', fontsize=14)
plt.xlabel('Odeme Tipi')
plt.ylabel('Gelir ($)')
plt.tight_layout()
plt.show()

### 5.4 Korelasyon Analizi ve Heatmap

In [ ]:
# Sayisal degiskenler arasindaki korelasyon
sayisal_sutunlar = ['Qty', 'UnitPrice', 'LineDiscount$', 'LineCost$', 'LineRevenue$', 
                    'StandardCost', 'MSRP', 'ShipCost$']

korelasyon = df[sayisal_sutunlar].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(korelasyon, annot=True, cmap='RdYlBu_r', center=0, fmt='.2f',
            square=True, linewidths=0.5)
plt.title('Korelasyon Matrisi (Heatmap)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot - Birim fiyat ve Gelir iliskisi
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df.sample(5000), x='UnitPrice', y='LineRevenue$', alpha=0.5)
plt.title('Birim Fiyat ve Satis Geliri Iliskisi', fontsize=14)
plt.xlabel('Birim Fiyat ($)')
plt.ylabel('Satis Geliri ($)')
plt.tight_layout()
plt.show()

### 5.5 Gruplandirma ve Aggregation Gorsellestirmeleri

In [ ]:
# Kategorilere gore toplam gelir - groupby ve agg kullanimi
kategori_gelir = df.groupby('Category')['LineRevenue$'].agg(['sum', 'mean', 'count']).reset_index()
kategori_gelir.columns = ['Kategori', 'Toplam Gelir', 'Ortalama Gelir', 'Satis Adedi']
kategori_gelir = kategori_gelir.sort_values('Toplam Gelir', ascending=False)

print("KATEGORILERE GORE GELIR ANALIZI")
print("=" * 60)
kategori_gelir

In [ ]:
# Kategori bazli toplam gelir grafigi
plt.figure(figsize=(14, 6))
sns.barplot(data=kategori_gelir, x='Kategori', y='Toplam Gelir', palette='rocket')
plt.title('Kategorilere Gore Toplam Satis Geliri', fontsize=14)
plt.xlabel('Kategori')
plt.ylabel('Toplam Gelir ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Kargo firmasi ve teslimat durumu capraz tablosu - pivot table
pivot_tablo = pd.pivot_table(df, 
                             values='OrderID', 
                             index='Carrier', 
                             columns='DeliveryStatus', 
                             aggfunc='count',
                             fill_value=0)

print("KARGO FIRMASI ve TESLIMAT DURUMU (PIVOT TABLE)")
print("=" * 60)
pivot_tablo

In [ ]:
# Pivot table heatmap gorsellestirmesi
plt.figure(figsize=(12, 6))
sns.heatmap(pivot_tablo, annot=True, fmt='d', cmap='YlGnBu')
plt.title('Kargo Firmasi ve Teslimat Durumu Capraz Tablosu', fontsize=14)
plt.xlabel('Teslimat Durumu')
plt.ylabel('Kargo Firmasi')
plt.tight_layout()
plt.show()

## 6. Istatistiksel Testler

Hipotez testleri yaparak verilerimiz hakkinda istatistiksel cikarimlar yapacagiz.

### 6.1 Normallik Testi (Shapiro-Wilk)

Verilerin normal dagilip dagilmadigini test ediyoruz.

**Hipotezler:**
- H0: Veriler normal dagilima sahiptir
- H1: Veriler normal dagilima sahip degildir

In [ ]:
# Shapiro-Wilk testi (kucuk orneklem icin)
# Buyuk veri setlerinde orneklem aliyoruz
orneklem = df['LineRevenue$'].sample(1000, random_state=42)

stat, p_value = shapiro(orneklem)

print("SHAPIRO-WILK NORMALLIK TESTI")
print("=" * 50)
print(f"Test Istatistigi: {stat:.4f}")
print(f"P-degeri: {p_value:.4f}")
print()

alpha = 0.05
if p_value > alpha:
    print(f"P-degeri ({p_value:.4f}) > alpha ({alpha})")
    print("Sonuc: H0 reddedilemez. Veriler normal dagilima sahip olabilir.")
else:
    print(f"P-degeri ({p_value:.4f}) < alpha ({alpha})")
    print("Sonuc: H0 reddedilir. Veriler normal dagilima sahip degildir.")

### 6.2 Iki Grup Karsilastirmasi (T-Testi)

Iki grubun ortalamalari arasinda anlamli fark olup olmadigini test ediyoruz.

In [ ]:
# Credit Card ve Debit Card odeme tiplerine gore gelir karsilastirmasi
credit_card_gelir = df[df['PaymentType'] == 'Credit Card']['LineRevenue$']
debit_card_gelir = df[df['PaymentType'] == 'Debit Card']['LineRevenue$']

stat, p_value = ttest_ind(credit_card_gelir.sample(1000, random_state=42), 
                          debit_card_gelir.sample(1000, random_state=42))

print("BAGIMSIZ ORNEKLEM T-TESTI")
print("Credit Card vs Debit Card Gelir Karsilastirmasi")
print("=" * 50)
print(f"Credit Card Ortalama Gelir: ${credit_card_gelir.mean():,.2f}")
print(f"Debit Card Ortalama Gelir: ${debit_card_gelir.mean():,.2f}")
print(f"\nT-istatistigi: {stat:.4f}")
print(f"P-degeri: {p_value:.4f}")
print()

alpha = 0.05
if p_value > alpha:
    print(f"Sonuc: Iki grup arasinda istatistiksel olarak anlamli fark yoktur (p > {alpha}).")
else:
    print(f"Sonuc: Iki grup arasinda istatistiksel olarak anlamli fark vardir (p < {alpha}).")

### 6.3 Coklu Grup Karsilastirmasi (ANOVA)

Ikiden fazla grubun ortalamalari arasinda anlamli fark olup olmadigini test ediyoruz.

In [ ]:
# Kargo firmalarina gore kargo ucreti karsilastirmasi (ANOVA)
kargo_firmalari = df['Carrier'].unique()
gruplar = [df[df['Carrier'] == kargo]['ShipCost$'].dropna() for kargo in kargo_firmalari]

stat, p_value = f_oneway(*gruplar)

print("TEK YONLU ANOVA TESTI")
print("Kargo Firmalarina Gore Kargo Ucreti Karsilastirmasi")
print("=" * 50)

for kargo in kargo_firmalari:
    ort = df[df['Carrier'] == kargo]['ShipCost$'].mean()
    print(f"{kargo} Ortalama Kargo Ucreti: ${ort:.2f}")

print(f"\nF-istatistigi: {stat:.4f}")
print(f"P-degeri: {p_value:.4f}")
print()

alpha = 0.05
if p_value > alpha:
    print(f"Sonuc: Gruplar arasinda istatistiksel olarak anlamli fark yoktur (p > {alpha}).")
else:
    print(f"Sonuc: En az iki grup arasinda istatistiksel olarak anlamli fark vardir (p < {alpha}).")

### 6.4 Korelasyon Testi (Pearson)

Iki sayisal degisken arasindaki dogrusal iliskinin gucunu ve yonunu test ediyoruz.

In [ ]:
# UnitPrice ve LineRevenue$ arasindaki korelasyon
korelasyon_katsayisi, p_value = pearsonr(df['UnitPrice'], df['LineRevenue$'])

print("PEARSON KORELASYON TESTI")
print("Birim Fiyat ve Satis Geliri Iliskisi")
print("=" * 50)
print(f"Korelasyon Katsayisi (r): {korelasyon_katsayisi:.4f}")
print(f"P-degeri: {p_value:.4e}")
print()

# Korelasyon yorumu
if abs(korelasyon_katsayisi) >= 0.7:
    guc = "guclu"
elif abs(korelasyon_katsayisi) >= 0.4:
    guc = "orta duzeyde"
else:
    guc = "zayif"

yon = "pozitif" if korelasyon_katsayisi > 0 else "negatif"

print(f"Yorum: Iki degisken arasinda {guc} {yon} iliski vardir.")

## 7. Veri On Isleme

Makine ogrenmesi modelleri icin veriyi hazirliyoruz.

### 7.1 Hedef Degisken Olusturma

Teslimat durumunu tahmin eden bir siniflandirma modeli kuracagiz. Bunun icin hedef degiskenimizi belirliyoruz.

In [ ]:
# Teslimat durumu dagilimini inceleyelim
print("TESLIMAT DURUMU DAGILIMI")
print("=" * 50)
print(df['DeliveryStatus'].value_counts())
print()
print(df['DeliveryStatus'].value_counts(normalize=True).round(4) * 100)

In [ ]:
# Hedef degisken: Teslimatin zamaninda yapilip yapilmadigi
# 'Delivered' -> 1 (Basarili), digerleri -> 0 (Basarisiz)
df['Basarili_Teslimat'] = (df['DeliveryStatus'] == 'Delivered').astype(int)

print("HEDEF DEGISKEN DAGILIMI")
print("=" * 50)
print(df['Basarili_Teslimat'].value_counts())
print()
print(f"Basarili Teslimat Orani: %{df['Basarili_Teslimat'].mean() * 100:.2f}")

### 7.2 Ozellik Secimi ve Veri Hazirlama

In [ ]:
# Model icin kullanilacak ozellikleri seciyoruz
# Kategorik degiskenler: Category, PaymentType, Carrier, ServiceLevel
# Sayisal degiskenler: Qty, UnitPrice, LineRevenue$, ShipCost$

# Sadece gerekli sutunlari seciyoruz ve eksik verileri temizliyoruz
model_df = df[['Category', 'PaymentType', 'Carrier', 'ServiceLevel',
               'Qty', 'UnitPrice', 'LineRevenue$', 'ShipCost$', 'Basarili_Teslimat']].copy()

# Eksik verileri kontrol edelim
print("EKSIK VERI KONTROLU")
print("=" * 50)
print(model_df.isnull().sum())

In [ ]:
# Eksik verileri temizliyoruz
model_df = model_df.dropna()
print(f"Temizlenen veri seti boyutu: {model_df.shape}")

### 7.3 Label Encoding

Kategorik degiskenleri sayisal degerlere donusturuyoruz.

In [ ]:
# Label Encoder nesnesi olusturuyoruz
le = LabelEncoder()

# Kategorik sutunlari donusturuyoruz
kategorik_sutunlar = ['Category', 'PaymentType', 'Carrier', 'ServiceLevel']

for sutun in kategorik_sutunlar:
    model_df[sutun + '_encoded'] = le.fit_transform(model_df[sutun])
    print(f"{sutun} donusturuldu.")
    print(f"Benzersiz degerler: {model_df[sutun].nunique()}")
    print()

In [ ]:
# Donusturulmus veri setini inceleyelim
model_df.head()

### 7.4 Ozellik ve Hedef Degiskenlerin Ayrilmasi

In [ ]:
# Bagimsiz degiskenler (X) ve bagimli degisken (y)
ozellik_sutunlari = ['Qty', 'UnitPrice', 'LineRevenue$', 'ShipCost$',
                     'Category_encoded', 'PaymentType_encoded', 
                     'Carrier_encoded', 'ServiceLevel_encoded']

X = model_df[ozellik_sutunlari]
y = model_df['Basarili_Teslimat']

print("OZELLIK MATRISI (X)")
print(f"Boyut: {X.shape}")
print(f"Sutunlar: {X.columns.tolist()}")
print()
print("HEDEF DEGISKEN (y)")
print(f"Boyut: {y.shape}")

### 7.5 Egitim ve Test Setlerine Ayirma

In [ ]:
# Veriyi %80 egitim, %20 test olarak ayiriyoruz
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print("VERI SETI BOLUNMESI")
print("=" * 50)
print(f"Egitim seti boyutu: {X_train.shape[0]} satir")
print(f"Test seti boyutu: {X_test.shape[0]} satir")
print(f"\nEgitim seti orani: %{len(X_train) / len(X) * 100:.0f}")
print(f"Test seti orani: %{len(X_test) / len(X) * 100:.0f}")

### 7.6 Olceklendirme (Scaling)

Farkli olceklerdeki degiskenleri standardlastiriyoruz.

In [ ]:
# StandardScaler ile olceklendirme
scaler = StandardScaler()

# Egitim seti uzerinde fit ve transform
X_train_scaled = scaler.fit_transform(X_train)

# Test seti uzerinde sadece transform (fit yok - veri sizintisini onlemek icin)
X_test_scaled = scaler.transform(X_test)

print("OLCEKLENDIRME TAMAMLANDI")
print("=" * 50)
print(f"Olceklendirme oncesi - Egitim seti ortalamasi: {X_train.mean().mean():.4f}")
print(f"Olceklendirme sonrasi - Egitim seti ortalamasi: {X_train_scaled.mean():.4f}")

## 8. Makine Ogrenmesi Modelleri

Farkli siniflandirma algoritmalari kullanarak modeller olusturacagiz.

### 8.1 Lojistik Regresyon

Lojistik regresyon, ikili siniflandirma problemleri icin kullanilan temel bir algoritmadir.

In [ ]:
# Lojistik Regresyon modeli
log_reg = LogisticRegression(solver='liblinear', random_state=42)
log_reg.fit(X_train_scaled, y_train)

# Tahmin
y_pred_log = log_reg.predict(X_test_scaled)

print("LOJISTIK REGRESYON SONUCLARI")
print("=" * 50)
print(f"Egitim Dogrulugu: %{log_reg.score(X_train_scaled, y_train) * 100:.2f}")
print(f"Test Dogrulugu: %{accuracy_score(y_test, y_pred_log) * 100:.2f}")

In [ ]:
# Karisiklik Matrisi (Confusion Matrix)
cm_log = confusion_matrix(y_test, y_pred_log)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_log, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Basarisiz', 'Basarili'],
            yticklabels=['Basarisiz', 'Basarili'])
plt.title('Lojistik Regresyon - Karisiklik Matrisi', fontsize=14)
plt.xlabel('Tahmin')
plt.ylabel('Gercek')
plt.tight_layout()
plt.show()

In [ ]:
# Siniflandirma Raporu
print("SINIFLANDIRMA RAPORU")
print("=" * 50)
print(classification_report(y_test, y_pred_log, target_names=['Basarisiz', 'Basarili']))

### 8.2 Karar Agaci (Decision Tree)

Karar agaclari, verileri karar kurallarina gore bolen yorumlanabilir bir algoritmadir.

In [ ]:
# Karar Agaci modeli
dt_model = DecisionTreeClassifier(random_state=42, max_depth=10)
dt_model.fit(X_train_scaled, y_train)

# Tahmin
y_pred_dt = dt_model.predict(X_test_scaled)

print("KARAR AGACI SONUCLARI")
print("=" * 50)
print(f"Egitim Dogrulugu: %{dt_model.score(X_train_scaled, y_train) * 100:.2f}")
print(f"Test Dogrulugu: %{accuracy_score(y_test, y_pred_dt) * 100:.2f}")

In [ ]:
# Karar Agaci - Karisiklik Matrisi
cm_dt = confusion_matrix(y_test, y_pred_dt)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Basarisiz', 'Basarili'],
            yticklabels=['Basarisiz', 'Basarili'])
plt.title('Karar Agaci - Karisiklik Matrisi', fontsize=14)
plt.xlabel('Tahmin')
plt.ylabel('Gercek')
plt.tight_layout()
plt.show()

In [ ]:
# Ozellik Onem Dereceleri
ozellik_onemleri = pd.DataFrame({
    'Ozellik': ozellik_sutunlari,
    'Onem': dt_model.feature_importances_
}).sort_values('Onem', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=ozellik_onemleri, x='Onem', y='Ozellik', palette='viridis')
plt.title('Karar Agaci - Ozellik Onem Dereceleri', fontsize=14)
plt.xlabel('Onem Derecesi')
plt.ylabel('Ozellik')
plt.tight_layout()
plt.show()

### 8.3 Random Forest (Rastgele Orman)

Random Forest, birden fazla karar agacini birlestiren guclu bir topluluk (ensemble) yontemidir.

In [ ]:
# Random Forest modeli
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train_scaled, y_train)

# Tahmin
y_pred_rf = rf_model.predict(X_test_scaled)

print("RANDOM FOREST SONUCLARI")
print("=" * 50)
print(f"Egitim Dogrulugu: %{rf_model.score(X_train_scaled, y_train) * 100:.2f}")
print(f"Test Dogrulugu: %{accuracy_score(y_test, y_pred_rf) * 100:.2f}")

In [ ]:
# Random Forest - Karisiklik Matrisi
cm_rf = confusion_matrix(y_test, y_pred_rf)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['Basarisiz', 'Basarili'],
            yticklabels=['Basarisiz', 'Basarili'])
plt.title('Random Forest - Karisiklik Matrisi', fontsize=14)
plt.xlabel('Tahmin')
plt.ylabel('Gercek')
plt.tight_layout()
plt.show()

In [ ]:
# Random Forest - Ozellik Onem Dereceleri
rf_ozellik_onemleri = pd.DataFrame({
    'Ozellik': ozellik_sutunlari,
    'Onem': rf_model.feature_importances_
}).sort_values('Onem', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=rf_ozellik_onemleri, x='Onem', y='Ozellik', palette='rocket')
plt.title('Random Forest - Ozellik Onem Dereceleri', fontsize=14)
plt.xlabel('Onem Derecesi')
plt.ylabel('Ozellik')
plt.tight_layout()
plt.show()

### 8.4 K-En Yakin Komsu (KNN)

KNN, bir veri noktasini en yakin k komsusuna gore siniflandiran basit bir algoritmadir.

In [ ]:
# KNN modeli
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

# Tahmin
y_pred_knn = knn_model.predict(X_test_scaled)

print("K-EN YAKIN KOMSU (KNN) SONUCLARI")
print("=" * 50)
print(f"Egitim Dogrulugu: %{knn_model.score(X_train_scaled, y_train) * 100:.2f}")
print(f"Test Dogrulugu: %{accuracy_score(y_test, y_pred_knn) * 100:.2f}")

In [ ]:
# KNN - Karisiklik Matrisi
cm_knn = confusion_matrix(y_test, y_pred_knn)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Purples',
            xticklabels=['Basarisiz', 'Basarili'],
            yticklabels=['Basarisiz', 'Basarili'])
plt.title('KNN - Karisiklik Matrisi', fontsize=14)
plt.xlabel('Tahmin')
plt.ylabel('Gercek')
plt.tight_layout()
plt.show()

## 9. Model Karsilastirmasi ve Degerlendirme

In [ ]:
# Tum modellerin sonuclarini karsilastiralim
model_sonuclari = pd.DataFrame({
    'Model': ['Lojistik Regresyon', 'Karar Agaci', 'Random Forest', 'KNN'],
    'Egitim Dogrulugu': [
        log_reg.score(X_train_scaled, y_train) * 100,
        dt_model.score(X_train_scaled, y_train) * 100,
        rf_model.score(X_train_scaled, y_train) * 100,
        knn_model.score(X_train_scaled, y_train) * 100
    ],
    'Test Dogrulugu': [
        accuracy_score(y_test, y_pred_log) * 100,
        accuracy_score(y_test, y_pred_dt) * 100,
        accuracy_score(y_test, y_pred_rf) * 100,
        accuracy_score(y_test, y_pred_knn) * 100
    ]
}).round(2)

print("MODEL KARSILASTIRMASI")
print("=" * 60)
print(model_sonuclari.to_string(index=False))

In [ ]:
# Model karsilastirma grafigi
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(model_sonuclari))
width = 0.35

bars1 = ax.bar(x - width/2, model_sonuclari['Egitim Dogrulugu'], width, label='Egitim', color='steelblue')
bars2 = ax.bar(x + width/2, model_sonuclari['Test Dogrulugu'], width, label='Test', color='coral')

ax.set_xlabel('Model')
ax.set_ylabel('Dogruluk (%)')
ax.set_title('Model Performans Karsilastirmasi', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(model_sonuclari['Model'])
ax.legend()
ax.set_ylim(0, 100)

# Degerleri barlarin uzerine yazalim
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:.1f}%', xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=9)

for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.1f}%', xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

### 9.1 ROC Egrisi ve AUC

In [ ]:
# ROC egrisini cizelim
plt.figure(figsize=(10, 8))

# Her model icin ROC egrisi
modeller = {
    'Lojistik Regresyon': (log_reg, 'blue'),
    'Karar Agaci': (dt_model, 'green'),
    'Random Forest': (rf_model, 'red'),
    'KNN': (knn_model, 'purple')
}

for model_adi, (model, renk) in modeller.items():
    # Olasilik tahminleri
    y_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # ROC egrisi degerleri
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    
    plt.plot(fpr, tpr, color=renk, label=f'{model_adi} (AUC = {auc:.3f})')

# Referans cizgisi (rastgele tahmin)
plt.plot([0, 1], [0, 1], 'k--', label='Rastgele Tahmin')

plt.xlabel('False Positive Rate (Yanlis Pozitif Orani)')
plt.ylabel('True Positive Rate (Dogru Pozitif Orani)')
plt.title('ROC Egrisi Karsilastirmasi', fontsize=14)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 9.2 Capraz Dogrulama (Cross Validation)

In [ ]:
# 5-Katli Capraz Dogrulama
print("5-KATLI CAPRAZ DOGRULAMA SONUCLARI")
print("=" * 60)

cv_sonuclari = []

for model_adi, (model, _) in modeller.items():
    cv_skorlari = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    cv_sonuclari.append({
        'Model': model_adi,
        'Ortalama': cv_skorlari.mean() * 100,
        'Std': cv_skorlari.std() * 100
    })
    print(f"{model_adi}:")
    print(f"  Skorlar: {[f'{s*100:.2f}%' for s in cv_skorlari]}")
    print(f"  Ortalama: %{cv_skorlari.mean() * 100:.2f} (+/- {cv_skorlari.std() * 100:.2f})")
    print()

cv_df = pd.DataFrame(cv_sonuclari)

In [ ]:
# Capraz dogrulama sonuclari grafigi
plt.figure(figsize=(10, 6))
plt.bar(cv_df['Model'], cv_df['Ortalama'], yerr=cv_df['Std'], capsize=5, color='teal', alpha=0.7)
plt.xlabel('Model')
plt.ylabel('Dogruluk (%)')
plt.title('5-Katli Capraz Dogrulama Sonuclari', fontsize=14)
plt.ylim(0, 100)
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

## 10. Sonuc ve Degerlendirme

In [ ]:
# En iyi modeli belirleyelim
en_iyi_model = model_sonuclari.loc[model_sonuclari['Test Dogrulugu'].idxmax()]

print("=" * 60)
print("PROJE SONUC RAPORU")
print("=" * 60)
print()
print("1. VERI SETI")
print(f"   - Toplam kayit sayisi: {len(df):,}")
print(f"   - Kullanilan ozellik sayisi: {len(ozellik_sutunlari)}")
print(f"   - Egitim seti: {len(X_train):,} kayit")
print(f"   - Test seti: {len(X_test):,} kayit")
print()
print("2. MODEL SONUCLARI")
for _, row in model_sonuclari.iterrows():
    print(f"   - {row['Model']}: Test Dogrulugu = %{row['Test Dogrulugu']:.2f}")
print()
print(f"3. EN IYI MODEL: {en_iyi_model['Model']}")
print(f"   - Test Dogrulugu: %{en_iyi_model['Test Dogrulugu']:.2f}")
print()
print("4. KULLANILAN TEKNIKLER")
print("   - Veri birlestirme (merge)")
print("   - Kesifsel veri analizi (EDA)")
print("   - Istatistiksel testler (Shapiro, T-test, ANOVA, Pearson)")
print("   - Gorsellestirmeler (Histogram, Boxplot, Heatmap, Scatter)")
print("   - Label Encoding")
print("   - StandardScaler")
print("   - Siniflandirma modelleri (Logistic, DT, RF, KNN)")
print("   - Capraz dogrulama (Cross Validation)")
print("   - ROC-AUC analizi")
print("=" * 60)

---

## Kaynaklar

- Python ile Veri Bilimi Ders Notlari
- Scikit-learn Dokumantasyonu: https://scikit-learn.org/
- Pandas Dokumantasyonu: https://pandas.pydata.org/
- Seaborn Dokumantasyonu: https://seaborn.pydata.org/

---